In [4]:
#STABLE DASH NETWORK GRAPH
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# Read CSV file
data = pd.read_csv("path/odsi_partner_data_raw.csv") #Download from github and replace with your filepath

# Create an empty graph
G = nx.Graph()

# Iterate through the data and add nodes and edges to the graph
for index, row in data.iterrows():
    odsi = row["ODSI"]
    has_partners = row["Partners"]
    list_of_partners = row["List of Partners"]

    # Add ODSI node to the graph
    G.add_node(odsi, node_type="ODSI", highlight=False)

    if has_partners.lower() == "yes":
        partners = list_of_partners.split(";")
        for partner in partners:
            partner = partner.strip()
            # Add partner node to the graph
            G.add_node(partner, node_type="partner", highlight=False)

            # Add edge between ODSI and partner
            G.add_edge(odsi, partner, highlight=False)

# Get positions of the nodes using a layout algorithm
pos = nx.spring_layout(G)

# Create the Dash app
app = JupyterDash(__name__)

def create_network_figure(G):
    # Extract node positions (x and y coordinates) for ODSIs and partners
    odsi_x = [pos[n][0] for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
    odsi_y = [pos[n][1] for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
    partner_x = [pos[n][0] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]
    partner_y = [pos[n][1] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]

    # Create edge traces for the graph
    edge_traces = []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_color = 'gold' if edge[2]['highlight'] else 'gray'
        edge_traces.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color=edge_color, width=1)))

    # Create node traces for ODSIs and partners
    odsi_colors = ['gold' if attr['highlight'] else 'blue' for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
    partner_colors = ['gold' if attr['highlight'] else 'red' for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]

    odsi_trace = go.Scatter(
        x=odsi_x, 
        y=odsi_y, 
        mode='markers', 
        hovertemplate='%{text}', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"],
        marker=dict(
            size=15, 
            color=odsi_colors, 
            line=dict(color='black', width=0.5)))
    partner_trace = go.Scatter(
        x=partner_x, 
        y=partner_y, 
        mode='markers', 
        hovertemplate='%{text}', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"], 
        marker=dict(
            size=10, 
            color=partner_colors, 
            line=dict(color='black', width=0.5)
        )
    )
    odsi_label_trace = go.Scatter(
        x=odsi_x, 
        y=odsi_y, 
        mode='text', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"], 
        textposition="bottom center", 
        hoverinfo='skip', 
        showlegend=False, 
        marker=dict(size=0, color='rgba(0,0,0,0)'), 
        textfont=dict(size=8)
    )
    partner_label_trace = go.Scatter(
        x=partner_x, 
        y=partner_y, 
        mode='text', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"], 
        textposition="bottom center", 
        hoverinfo='skip', 
        showlegend=False, 
        marker=dict(size=0, color='rgba(0,0,0,0)'), 
        textfont=dict(size=8)
    )

    # Create the updated interactive figure
    updated_fig = go.Figure(edge_traces + [odsi_trace, partner_trace, odsi_label_trace, partner_label_trace])

    # Set the axis properties and layout
    updated_fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
    updated_fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)
    updated_fig.update_layout(showlegend=False, hovermode='closest', margin=dict(l=0, r=0, b=0, t=0))

    return updated_fig

fig = create_network_figure(G)

app.layout = html.Div([
    dcc.Graph(id='network-graph', figure=fig),
    html.Div(id='click-output')
])

@app.callback(
    Output('network-graph', 'figure'),
    [Input('network-graph', 'clickData')])
def update_graph_on_click(clickData):
    if clickData:
        clicked_node = clickData['points'][0]['text']
        
        # Highlight the clicked node
        for node, attr in G.nodes(data=True):
            if node == clicked_node:
                attr['highlight'] = True
            else:
                attr['highlight'] = False
        
        # Highlight connected edges and nodes
        for u, v, attr in G.edges(data=True):
            if u == clicked_node or v == clicked_node:
                attr['highlight'] = True
            else:
                attr['highlight'] = False
        
        # Update the figure
        updated_fig = create_network_figure(G)
        return updated_fig
    else:
        return fig



if __name__ == '__main__':
    app.run_server(mode='jupyterlab')



Dash is running on http://127.0.0.1:8050/

